In [0]:
import cv2 #pip install opencv.python

In [0]:
import matplotlib.pyplot as plt #pip install matplotlib

In [0]:
import pyttsx3

In [0]:
config_file='ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model='frozen_inference_graph.pb'

In [0]:
model=cv2.dnn_DetectionModel(frozen_model,config_file)

In [0]:
classLabels=[]
file_name='coco_labels.txt'
with open(file_name,'rt')as fpt:
    classLabels=fpt.read().rstrip('\n').split('\n')
    print(classLabels)

In [0]:
print(len(classLabels))

In [0]:
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)

#read an image

In [0]:
import cv2
import matplotlib.pyplot as plt
img=cv2.imread('hi.png')
plt.imshow(img)

In [0]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [0]:
ClassIndex,confidence,bbox=model.detect(img,confThreshold=0.5)

In [0]:
print(ClassIndex)

In [0]:
font_scale=3
font=cv2.FONT_HERSHEY_PLAIN
for ClassInd,conf,boxes in zip(ClassIndex.flatten(),confidence.flatten(),bbox):
    cv2.rectangle(img,boxes,(255,0,0),2)
    cv2.putText(img,classLabels[ClassInd-1],(boxes[0]+10,boxes[1]+40),font,fontScale=font_scale,color=(0,255,0),thickness=3)

In [0]:
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [0]:
import cv2
import os

# Function to check if file exists
def file_exists(file_path):
    return os.path.isfile(file_path)

# Open video file or camera stream
video_path = r"C:\Users\DELL\Desktop\project7\object.mp4"
if file_exists(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file: {video_path}")
        print("Trying camera...")
        cap = cv2.VideoCapture(0)  # Try to open the default camera
else:
    print(f"Video file not found: {video_path}")
    print("Trying camera...")
    cap = cv2.VideoCapture(0)  # Try to open the default camera

if not cap.isOpened():
    raise IOError('Cannot open video source')

# Load the object detection model
# Assuming you are using a pre-trained model (e.g., MobileNet, YOLO, etc.)
# For example, using MobileNet SSD for object detection:
# model = cv2.dnn_DetectionModel('path_to_model')

# Check if the model is loaded
try:
    model
except NameError:
    raise Exception("Object detection model not loaded. Please load the model before detection.")

# Font settings
font_scale = 3
font = cv2.FONT_HERSHEY_PLAIN

frame_count = 0
while True:
    # Read frame from the video capture
    ret, frame = cap.read()
    
    # If we've reached the end of the video, reset to the beginning
    if not ret or frame is None:
        print(f"Reached end of video after {frame_count} frames. Restarting...")
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        frame_count = 0
        continue
    
    frame_count += 1
    
    # Perform object detection if a valid frame is captured
    try:
        ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
    except Exception as e:
        print(f"Object detection failed: {e}")
        break
    
    print(ClassIndex)  # Print detected class indexes
    
    # If objects are detected, draw bounding boxes and labels
    if len(ClassIndex) != 0:
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                # Increase the size of the bounding box
                x, y, w, h = boxes
                expansion = 20  # Increase this value to make the rectangle larger
                x1 = max(0, x - expansion)
                y1 = max(0, y - expansion)
                x2 = min(frame.shape[1], x + w + expansion)
                y2 = min(frame.shape[0], y + h + expansion)
                
                # Draw rectangle around detected object with increased size
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
                # Put text label on the detected object
                cv2.putText(frame, classLabels[ClassInd - 1], 
                            (x1 + 10, y1 + 40), 
                            font, 
                            fontScale=font_scale, 
                            color=(0, 255, 0), 
                            thickness=3)
    
    # Display the frame with detected objects
    cv2.imshow('project7', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
print(f"Processed {frame_count} frames before exiting.")

In [0]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Load class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Open video file or camera stream
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Error: Unable to open video file. Trying camera...")
    cap = cv2.VideoCapture(0)  # Try to open the default camera
if not cap.isOpened():
    raise IOError('Cannot open webcam')

# Font settings
font_scale = 1.5  # Increased from 0.5 to 1.5
font = cv2.FONT_HERSHEY_SIMPLEX  # Changed from FONT_HERSHEY_PLAIN for better visibility
font_thickness = 2  # Added font thickness

while True:
    # Read frame from the video capture
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Failed to capture frame. Exiting...")
        break

    height, width, channels = frame.shape

    # Detect objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Information to display
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Green
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            
            # Calculate text size and position
            text = f"{label} {confidence:.2f}"
            (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
            text_offset_x = x
            text_offset_y = y - 5
            box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
            cv2.rectangle(frame, box_coords[0], box_coords[1], color, cv2.FILLED)
            cv2.putText(frame, text, (text_offset_x, text_offset_y), font, font_scale, (0, 0, 0), font_thickness)

    # Display the frame with detected objects
    cv2.imshow('Object Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()